In [1]:
import pickle
from layers import *

2024-04-22 14:56:01.241431: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-22 14:56:01.241676: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-22 14:56:01.244076: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-22 14:56:01.272267: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 14:56:01.864123: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

In [36]:
x = xin = Input((3,)) # input could be a multidim vector (e.g., (5,3) you have have 5 embeddings with dims of 3)

# y = BatchNorm(x, 0.1)
# y = SkipConn(y, 0.1, 5, x) # skip conn from input
# y = Attn(y, 0.1, 3)
# y = Dense(y, 0.1, 5)
# y = Conv(y, 0.1, 3)
# z = BatchNorm(y, 0.1)
# y = Dense(z, 0.1, 4)
# y = SkipConn(y, 0.1, 4, z)

x = Attn(x, 0.1, 3)
y=x

model = y
model([
    [1,2, 4]
])

<tf.Tensor: shape=(1, 3, 3), dtype=float32, numpy=
array([[[ 0.08551089, -0.17758447,  1.1125791 ],
        [ 0.09310377, -0.16355988,  1.222843  ],
        [ 0.09349057, -0.16271539,  1.2285266 ]]], dtype=float32)>

In [37]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('model.pkl', 'rb') as f:
    model2 = pickle.load(f)

AttributeError: Can't pickle local object 'Layer._initialize_tracker.<locals>.<lambda>'

In [8]:
model2([
    [1,2, 4]
])

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[1.0853643, 1.3232546, 1.3385601, 1.0774878]], dtype=float32)>